In [72]:
from implementations import *
from helpers import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime




## Load the data ###

In [73]:
data_path='./dataset_to_release/'
train_data_path="./dataset_to_release/x_train.csv"
test_data_path="./dataset_to_release/x_test.csv"

In [74]:
x_train, x_test, y_train, train_ids, test_ids=load_csv_data_all(data_path, sub_sample=False)

In [11]:
# train_data_path="./dataset/x_train.csv"
# train_label_path="./dataset/y_train.csv"
# train_data=pd.read_csv(train_data_path)
# train_label=pd.read_csv(train_label_path)


In [12]:
# load the data points
# xb=load_csv_data(train_data_path,sub_sample=False)

In [4]:
# we load the labels 
# yb=load_csv_data_labels(train_label_path, sub_sample=False)

In [5]:

# y = np.expand_dims(yb, axis=1)

In [6]:
# Extract the column names from the x_train.csv and convert it into a list



## Data Pre-Processing

In [75]:
# Handeling the missing values

def replace_nan_by_mean(data):
    ''' function that handels the missing values by replacing them with the column means'''
    nan_indices = np.isnan(data)
    column_means = np.nanmean(data, axis=0)
    data[nan_indices] = np.take(column_means, np.where(nan_indices)[1])
    return data


data_train=replace_nan_by_mean(x_train)


In [8]:
## Verifying that the nan values have been successfully removed
# matrix=np.isnan(data_train)
# num_true=np.count_nonzero(matrix)
# num_false = matrix.size - num_true
# print(num_true)

In [76]:
x_train.shape

(328135, 321)

In [77]:
# Data filtering: we only keep relevent features

def filtering(data, data_path):
    columns=extract_first_line(data_path).split(',')
    columns.pop(0)
    columns_to_keep=[]
    for c in columns:
        if c.startswith('_'):
            columns_to_keep.append(c)
    indices_to_keep = [columns.index(c) for c in columns_to_keep]
    data_f=data[:, indices_to_keep]
    return(data_f)



In [78]:
data_train_filtered=filtering(data_train, train_data_path)


## Training

In [79]:
# standardization of the data
def standardize(data):
    mean=np.mean(data, axis=0)
    std=np.std(data, axis=0)
    return((data - mean) / (std))

In [80]:
data_standardized = standardize(data_train_filtered)

In [11]:
#split the test set in two


def split_data(x, y, ratio, seed=1):
    """
    split the dataset based on the split ratio. If ratio is 0.8
    you will have 80% of your data set dedicated to training
    and the rest dedicated to testing. If ratio times the number of samples is not round
    you can use np.floor. Also check the documentation for np.random.permutation,
    it could be useful.

    Args:
        x: numpy array of shape (N,), N is the number of samples.
        y: numpy array of shape (N,).
        ratio: scalar in [0,1]
        seed: integer.

    Returns:
        x_tr: numpy array containing the train data.
        x_te: numpy array containing the test data.
        y_tr: numpy array containing the train labels.
        y_te: numpy array containing the test labels.
    """
    N=int(ratio*len(x))
    # set seed
    np.random.seed(seed)
    # split the data based on the given ratio: 
    shuffled_data=np.random.permutation(x)
    #print(shuffled_data)
    np.random.seed(seed)
    shuffled_labels=np.random.permutation(y)
    #print(shuffled_labels)
    x_tr=shuffled_data[:N] #train data
    x_te=shuffled_data[N:] #test data
    y_tr=shuffled_labels[:N]#train labels
    y_te=shuffled_labels[N:]# test labels

    return(x_tr,x_te, y_tr, y_te)

In [12]:
#x_tr,x_te, y_tr, y_te=split_data(data_standardized, y_train, ratio=0.8)
#y_te=np.expand_dims(y_te, 1)
#y_tr=np.expand_dims(y_tr, 1)

In [13]:
#print(y_tr.shape, x_tr.shape)

(262508, 1) (262508, 75)


In [85]:
y_train=np.expand_dims(y_train, 1)

In [86]:
# Binary classification using logistic regression

max_iters = 10000
gamma = 0.5

 # build tx
tx_tr = np.c_[np.ones((y_train.shape[0], 1)), data_standardized]
initial_w=np.zeros((tx_tr.shape[1], 1))


In [87]:
y_train = np.where(y_train == -1, 0, y_train)

In [88]:
y_train

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [20]:
#grad=calculate_log_likelihood_gradient(y_train,tx_tr,initial_w)


In [21]:
# w=initial_w-gamma*grad
# print(np.min(w), np.max(w))

-0.20559754369390645 0.03185866857918228


In [22]:
# loss=calculate_log_likelihood_loss(y_train,tx_tr,initial_w)
# print(loss)

0.6031907630182123


In [89]:
w,loss= logistic_regression(y_train, tx_tr, initial_w, max_iters, gamma=0.5)

Current iteration=0, loss=0.6030287143641271
Current iteration=100, loss=0.2508686434767057
Current iteration=200, loss=0.24930194108636033
Current iteration=300, loss=0.24902538226118237
Current iteration=400, loss=0.24894621240064338
Current iteration=500, loss=0.24891299793724914
Current iteration=600, loss=0.24889282517295436
Current iteration=700, loss=0.2488773650326659
Current iteration=800, loss=0.24886423950609554
Current iteration=900, loss=0.24885264188892714
Current iteration=1000, loss=0.24884221835471887
Current iteration=1100, loss=0.2488327645612468
Current iteration=1200, loss=0.2488241372356895
Current iteration=1300, loss=0.2488162252318069
Current iteration=1400, loss=0.24880893813753485
Current iteration=1500, loss=0.24880220064664463
Current iteration=1600, loss=0.24879594916608747
Current iteration=1700, loss=0.2487901294876123
Current iteration=1800, loss=0.24878469506948184
Current iteration=1900, loss=0.24877960571607194
Current iteration=2000, loss=0.24877482

In [31]:

# lambda_ = 0.0005
# w_reg,loss_reg= reg_logistic_regression(y_train, tx_tr, lambda_, initial_w, max_iters, gamma)

Current iteration=0, loss=0.6031907630182123
Current iteration=100, loss=0.2520887951460211
Current iteration=200, loss=0.25039224509073077
Current iteration=300, loss=0.2500521566247474
Current iteration=400, loss=0.24993664541267355
Current iteration=500, loss=0.2498842695901215
Current iteration=600, loss=0.24985524379278526
Current iteration=700, loss=0.24983655399036112
Current iteration=800, loss=0.24982309818208762
Current iteration=900, loss=0.24981262703336746
Current iteration=1000, loss=0.2498040556914796
Current iteration=1100, loss=0.24979681368238313
Current iteration=1200, loss=0.24979057224301943
Current iteration=1300, loss=0.24978512356961943
Current iteration=1400, loss=0.249780324917527
Current iteration=1500, loss=0.2497760714712832
Current iteration=1600, loss=0.2497722824081014
Current iteration=1700, loss=0.24976889323836618
Current iteration=1800, loss=0.24976585127206166
Current iteration=1900, loss=0.249763112732196
Current iteration=2000, loss=0.249760640794

## Test and Accuracy

In [32]:
tx_te=np.c_[np.ones((y_te.shape[0], 1)), x_te]

In [33]:
y_te=np.where(y_te==-1, 0, y_te)

In [34]:
def apply_model(test, model):
    pred=(sigmoid(test.dot(model))>=0.5).astype(int)
    return(pred)
    

In [35]:
y_pred=apply_model(tx_te, w_reg)

In [36]:
#Calculate accuracy
correct_predictions = np.sum(y_pred == y_te)
total_samples = len(y_te)
accuracy = correct_predictions / total_samples

In [37]:
print(accuracy)

0.9106312950462462


## Test

In [91]:
xt=replace_nan_by_mean(x_test)

In [92]:
x_test.shape

(109379, 321)

In [104]:
w

array([[-2.98439905e+00],
       [-6.71278927e-04],
       [ 2.87138546e-03],
       [ 1.59538068e-02],
       [ 1.53392851e-03],
       [ 3.76353710e-02],
       [ 2.07579264e-02],
       [-7.23643419e-03],
       [ 4.91416283e-04],
       [ 4.91416283e-04],
       [-9.81921163e-03],
       [ 4.61033861e-02],
       [-3.26303386e-02],
       [ 5.86030010e-03],
       [ 2.26074356e-01],
       [ 3.89504974e-02],
       [ 2.04549791e-01],
       [-1.29211982e-01],
       [ 1.26911369e-01],
       [ 1.84269871e-02],
       [ 7.67404993e-02],
       [-1.09742873e-01],
       [-1.50566704e-01],
       [-1.62424355e-01],
       [ 1.74814008e-01],
       [ 1.21415129e-02],
       [ 4.29799725e-02],
       [ 1.09373738e-02],
       [-7.62468874e-02],
       [ 2.45340791e-02],
       [ 2.62838941e-01],
       [-8.93709087e-02],
       [ 7.63636773e-01],
       [ 5.83425844e-02],
       [ 3.36706373e-02],
       [ 1.34704159e-01],
       [-6.52823855e-02],
       [-2.50762842e-02],
       [-6.4

In [93]:
xt_filtered=filtering(xt, test_data_path)

In [94]:
xt_filtered.shape

(109379, 75)

In [95]:

xt_filtered_standardized=standardize(xt_filtered)
xtest=np.c_[np.ones((xt.shape[0], 1)), xt_filtered_standardized]


In [96]:
xtest.shape

(109379, 76)

In [106]:
predictions=apply_model(xtest, w)
predictions=np.where(predictions==0,-1, predictions)

In [107]:
predictions


array([[-1],
       [-1],
       [-1],
       ...,
       [-1],
       [-1],
       [-1]])

In [108]:
create_csv_submission(test_ids, predictions, 'predictions_v1_with_logistic_regression.csv')